# Shift maliciousness (3dshapes)

**Author**: Maleakhi A. Wijaya  
**Description**: Inspect the maliciousness of different type and intensity of shifts.

In [1]:
# Load utilities functions
%run ../../scripts/constants.py
%run ../../scripts/3dshapes_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py
from sklearn.metrics import accuracy_score

In [2]:
## Random seed
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load dataset

In [3]:
files_dir = "../../data/3dshapes.h5"
# index 0 = image category
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split_3dshapes(files_dir, 70000, DatasetTask.Task1, 
                                                                               train_size=0.80, class_index=3)

Training samples: 56000
Testing samples: 14000


In [4]:
n_classes = len(np.unique(np.concatenate([y_train, y_test])))
concept_names = SHAPES3D_CONCEPT_NAMES
concept_values = [len(np.unique(np.concatenate([c_train[:, i], c_test[:, i]]))) for i in range(c_train.shape[1])]

# Split training into validation set as well 
X_train, X_valid = X_train[:40000], X_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]
c_train, c_valid = c_train[:40000], c_train[40000:]

In [5]:
# Load adversarial samples (we will need this for adversarial
# shift).
adv_samples = np.load("../../data/adversarial_samples/X_adversarial_3dshapes.npy")

In [6]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

## Load models & original prediction results

In [7]:
path = "../../models/end_to_end_3dshapes_task1"
end_to_end_model = tf.keras.models.load_model(path)

path = "../../models/multitask_3dshapes"
itc_model = tf.keras.models.load_model(path)

In [10]:
# Evaluate model (FFNN)
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 1.0


In [11]:
# Evaluate model (concept bottleneck model)
for i, pred in enumerate(itc_model.predict(X_test)):
    print("*"*20, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")

******************** Model: floor ********************
Accuracy: 1.0
******************** Model: wall ********************
Accuracy: 1.0
******************** Model: object ********************
Accuracy: 1.0
******************** Model: scale ********************
Accuracy: 1.0
******************** Model: shape ********************
Accuracy: 1.0
******************** Model: orientation ********************
Accuracy: 1.0


## Gaussian shift

In [12]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_gaussian = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_gaussian_shift(X_shifted, y_test, shift_intensity, 1.0)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_gaussian.append(X_shifted)

In [13]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_gaussian, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 1.0
******************** medium ********************
Accuracy: 1.0
******************** large ********************
Accuracy: 0.7172142857142857


In [14]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_gaussian, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** medium ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** large ********************
********** Model: floor **********
Accuracy: 0.9168571428571428
********** Model: wall **********
Accuracy: 0.7029285714285715
********** Model: object **********
Accuracy: 0.5894285714285714
********** Model: scale **********
Accuracy: 0.62371428571428

## Knockout shift

In [15]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_ko = []
y_ko = []
c_ko = []

for shift_intensity in shift_intensities:
    X_shifted, y_shifted, c_shifted = deepcopy(X_test_flatten), deepcopy(y_test), deepcopy(c_test)
    X_shifted, y_shifted, c_shifted = apply_ko_shift(X_shifted, y_shifted, c_shifted, shift_intensity, cl=MAJORITY)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_ko.append(X_shifted)
    y_ko.append(y_shifted)
    c_ko.append(c_shifted)

In [16]:
# Print accuracy score (end-to-end)
for x, y, c, intensity_str in zip(X_ko, y_ko, c_ko, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y, y_pred)}")

******************** small ********************
Accuracy: 1.0
******************** medium ********************
Accuracy: 1.0
******************** large ********************
Accuracy: 1.0


In [17]:
# Print accuracy score (cbm)
for x, y, c, intensity_str in zip(X_ko, y_ko, c_ko, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** medium ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** large ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********

## Concept shift

In [18]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_cs = []
y_cs = []
c_cs = []

for shift_intensity in shift_intensities:
    X_shifted, y_shifted, c_shifted = deepcopy(X_test_flatten), deepcopy(y_test), deepcopy(c_test)
    X_shifted, y_shifted, c_shifted = apply_concept_shift(X_shifted, y_shifted, c_shifted, 2, shift_intensity, cl=MAJORITY)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_cs.append(X_shifted)
    y_cs.append(y_shifted)
    c_cs.append(c_shifted)

In [19]:
# Print accuracy score (end-to-end)
for x, y, c, intensity_str in zip(X_cs, y_cs, c_cs, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y, y_pred)}")

******************** small ********************
Accuracy: 1.0
******************** medium ********************
Accuracy: 1.0
******************** large ********************
Accuracy: 1.0


In [20]:
# Print accuracy score (cbm)
for x, y, c, intensity_str in zip(X_cs, y_cs, c_cs, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** medium ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********** Model: orientation **********
Accuracy: 1.0

******************** large ********************
********** Model: floor **********
Accuracy: 1.0
********** Model: wall **********
Accuracy: 1.0
********** Model: object **********
Accuracy: 1.0
********** Model: scale **********
Accuracy: 1.0
********** Model: shape **********
Accuracy: 1.0
********

## Image shift

In [8]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_img = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_img_shift(X_shifted, y_test, ORIGINAL_SHAPE,
                                   shift_intensity, 1.0, ShiftType.All)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_img.append(X_shifted)

In [9]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_img, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 0.25485714285714284
******************** medium ********************
Accuracy: 0.14535714285714285
******************** large ********************
Accuracy: 0.12792857142857142


In [10]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_img, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: floor **********
Accuracy: 0.999
********** Model: wall **********
Accuracy: 0.8791428571428571
********** Model: object **********
Accuracy: 0.9256428571428571
********** Model: scale **********
Accuracy: 0.28535714285714286
********** Model: shape **********
Accuracy: 0.7187142857142857
********** Model: orientation **********
Accuracy: 0.30378571428571427

******************** medium ********************
********** Model: floor **********
Accuracy: 0.8062142857142857
********** Model: wall **********
Accuracy: 0.403
********** Model: object **********
Accuracy: 0.49457142857142855
********** Model: scale **********
Accuracy: 0.1417857142857143
********** Model: shape **********
Accuracy: 0.2934285714285714
********** Model: orientation **********
Accuracy: 0.07671428571428572

******************** large ********************
********** Model: floor **********
Accuracy: 0.4575714285714286
********** Model: wall ********

## Adversarial Shift

In [11]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_adv = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_adversarial_shift(X_shifted, y_test, adv_samples,
                                   range(len(X_shifted)), shift_intensity)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_adv.append(X_shifted)

In [12]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_adv, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 0.9353571428571429
******************** medium ********************
Accuracy: 0.6747142857142857
******************** large ********************
Accuracy: 0.3485


In [13]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_adv, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SHAPES3D_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: floor **********
Accuracy: 0.9990714285714286
********** Model: wall **********
Accuracy: 0.9970714285714286
********** Model: object **********
Accuracy: 0.9913571428571428
********** Model: scale **********
Accuracy: 0.9503571428571429
********** Model: shape **********
Accuracy: 0.9872857142857143
********** Model: orientation **********
Accuracy: 0.9914285714285714

******************** medium ********************
********** Model: floor **********
Accuracy: 0.9955714285714286
********** Model: wall **********
Accuracy: 0.9879285714285714
********** Model: object **********
Accuracy: 0.9519285714285715
********** Model: scale **********
Accuracy: 0.7466428571428572
********** Model: shape **********
Accuracy: 0.9318571428571428
********** Model: orientation **********
Accuracy: 0.9625

******************** large ********************
********** Model: floor **********
Accuracy: 0.9894285714285714
********** Model: wal